In [15]:
import numpy as np
import pandas as pd

In [16]:
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.tree import DecisionTreeClassifier

In [17]:
df = pd.read_csv('train.csv')

In [18]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [19]:
# drop columns passengerid, name, ticket, cabin
# drop these because they are not proportional to any death
df.drop(columns=['PassengerId','Name','Ticket','Cabin'],inplace=True)


In [20]:
#train test split 
X_train, X_test, y_train, y_test = train_test_split(df.drop(columns=['Survived']),
                                                           df['Survived'],
                                                           test_size = 0.3,
                                                           random_state=42)
X_train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
445,1,male,4.0,0,2,81.8583,S
650,3,male,NaN,0,0,7.8958,S
172,3,female,1.0,1,1,11.1333,S
450,2,male,36.0,1,2,27.7500,S
314,2,male,43.0,1,1,26.2500,S


In [22]:
print(y_train.sample(8))

391    1
740    1
421    0
335    0
613    0
372    0
749    0
314    0
Name: Survived, dtype: int64


In [23]:
#step1 impute missing values

In [24]:
trf1 = ColumnTransformer([
    ('impute_age', SimpleImputer(), [2]),
    ('impute_embarked', SimpleImputer(strategy='most_frequent'), [6])
    
], remainder='passthrough')

In [27]:
# onehot enoding for nominal categorical values
#sex embarked both will work together , we'll specify there index rather names, to avoid converting numpy
trf2= ColumnTransformer([
    ('ohe_sex_embarked', OneHotEncoder(sparse_output=False, handle_unknown='ignore'), [1,6])
],remainder = 'passthrough')
#remainder passthrough passes remainder columns without changing them


In [29]:
#scaling
trf3=ColumnTransformer([
    ('scale', MinMaxScaler(),slice(0,10))
])

In [30]:
#feature selection 
trf4=SelectKBest(score_func=chi2,k=8)

In [32]:
#train the model
trf5 = DecisionTreeClassifier()

In [33]:
#create pipeline
pipe=Pipeline([
    ('trf1',trf1),
    ('trf2',trf2),
    ('trf3',trf3),
    ('trf4',trf4),
    ('trf5',trf5),
])

In [39]:
#pipeline vs make_pipeline
#pipeline requires names vs make_pipelines don't
pipe2=make_pipeline(trf1,trf2,trf3,trf4,trf5)

In [40]:
#train
pipe.fit(X_train,y_train)

Pipeline(steps=[('columntransformer-1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('impute_embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('columntransformer-2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_embarked',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  [1, 6])])),
                ('columntransformer-3',
                 ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('selectkbest',
                 SelectKBest(k=8,
                             score_func=<function chi2 at 0x000001961A156C00>)),
                ('decisiontreeclassifier', DecisionTreeClassifier())])

In [41]:
#when we're not training a model using pipeline call fit-transform otherwise pipe.fit

In [42]:
#pipline exploration
pipe.named_steps
# the steps performed

{'columntransformer-1': ColumnTransformer(remainder='passthrough',
                   transformers=[('impute_age', SimpleImputer(), [2]),
                                 ('impute_embarked',
                                  SimpleImputer(strategy='most_frequent'),
                                  [6])]),
 'columntransformer-2': ColumnTransformer(remainder='passthrough',
                   transformers=[('ohe_sex_embarked',
                                  OneHotEncoder(handle_unknown='ignore',
                                                sparse_output=False),
                                  [1, 6])]),
 'columntransformer-3': ColumnTransformer(transformers=[('scale', MinMaxScaler(), slice(0, 10, None))]),
 'selectkbest': SelectKBest(k=8, score_func=<function chi2 at 0x000001961A156C00>),
 'decisiontreeclassifier': DecisionTreeClassifier()}

In [51]:
#nae of cloumn transformer
pipe.named_steps.keys()

dict_keys(['columntransformer-1', 'columntransformer-2', 'columntransformer-3', 'selectkbest', 'decisiontreeclassifier'])

In [52]:
#what's inside first column transformer
pipe.named_steps['columntransformer-1']

ColumnTransformer(remainder='passthrough',
                  transformers=[('impute_age', SimpleImputer(), [2]),
                                ('impute_embarked',
                                 SimpleImputer(strategy='most_frequent'),
                                 [6])])

In [53]:
#how many transformers inside first column transformer
pipe.named_steps['columntransformer-1'].transformers_

[('impute_age', SimpleImputer(), [2]),
 ('impute_embarked', SimpleImputer(strategy='most_frequent'), [6]),
 ('remainder',
  FunctionTransformer(accept_sparse=True, check_inverse=False,
                      feature_names_out='one-to-one'),
  [0, 1, 3, 4, 5])]

In [54]:
#accessing transformer at index 0 [impute age]
pipe.named_steps['columntransformer-1'].transformers_[0]

('impute_age', SimpleImputer(), [2])

In [55]:
#access index 1 thing [simpleimputer]
pipe.named_steps['columntransformer-1'].transformers_[0][1]

SimpleImputer()

In [56]:
#access statistics of this imputer age
pipe.named_steps['columntransformer-1'].transformers_[0][1].statistics_

array([29.25635271])

In [61]:
# exploring embarked 
pipe.named_steps['columntransformer-1'].transformers_[1][1].statistics_

array(['S'], dtype=object)

In [62]:
# s is most frequent according to imputer

In [63]:
#cross validation using pipeline
from sklearn.model_selection import cross_val_score
cross_val_score(pipe, X_train,y_train, cv=3, scoring='accuracy').mean()

0.6516552087204261

In [70]:
#gridsearch using pipeline
params={
    'decisiontreeclassifier__max_depth':[1,2,3,4,5,None]
}

In [71]:
from sklearn.model_selection import GridSearchCV
grid = GridSearchCV(pipe, params, cv=3, scoring='accuracy')
grid.fit(X_train,y_train)

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('columntransformer-1',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('impute_age',
                                                                         SimpleImputer(),
                                                                         [2]),
                                                                        ('impute_embarked',
                                                                         SimpleImputer(strategy='most_frequent'),
                                                                         [6])])),
                                       ('columntransformer-2',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('ohe_sex_embarked',
                                                                         OneHotEncoder(handle_unkno...,
                                                                                       sparse_output=False),
                                                                         [1,
                                                                          6])])),
                                       ('columntransformer-3',
                                        ColumnTransformer(transformers=[('scale',
                                                                         MinMaxScaler(),
                                                                         slice(0, 10, None))])),
                                       ('selectkbest',
                                        SelectKBest(k=8,
                                                    score_func=<function chi2 at 0x000001961A156C00>)),
                                       ('decisiontreeclassifier',
                                        DecisionTreeClassifier())]),
             param_grid={'decisiontreeclassifier__max_depth': [1, 2, 3, 4, 5,
                                                               None]},
             scoring='accuracy')

In [72]:
grid.best_score_

0.6516552087204261

In [74]:
grid.best_params_

{'decisiontreeclassifier__max_depth': 5}